---
title: "Demographic Data"
format: html
jupyter: python3
execute:
  echo: false
  output: false
---


## Overview

This page presents demographic information for all incorporated cities in California, including:

- Population totals  
- Racial and ethnic composition  
- Median household income  

The data is drawn from public datasets and has been cleaned for clarity and consistency. Use the map, table, and charts below to explore.


## Demographic Data: All Cities

Use the table below to explore demographic data by city.

<!-- 🔍 Search bar -->
<input
  type="text"
  id="searchInput"
  onkeyup="filterTable()"
  placeholder="Search the table..."
  style="margin-bottom: 4px; padding: 10px; width: 100%; max-width: 400px;"
>

<script>
let currentPage = 1;
const rowsPerPage = 10;

function filterTable() {
  currentPage = 1;
  renderTable();
}

function sortTable(n) {
  const table = document.getElementById("mytable");
  const rows = Array.from(table.querySelectorAll("tbody tr"));
  const dir = table.dataset.sortDir === "asc" ? "desc" : "asc";
  table.dataset.sortDir = dir;
  table.dataset.sortCol = n;

  rows.sort((a, b) => {
    const cellA = a.children[n].textContent.trim().toLowerCase();
    const cellB = b.children[n].textContent.trim().toLowerCase();
    if (!isNaN(cellA) && !isNaN(cellB)) {
      return dir === "asc" ? cellA - cellB : cellB - cellA;
    }
    return dir === "asc"
      ? cellA.localeCompare(cellB)
      : cellB.localeCompare(cellA);
  });

  const tbody = table.querySelector("tbody");
  tbody.innerHTML = "";
  rows.forEach(row => tbody.appendChild(row));

  renderTable();
}

function renderTable() {
  const input = document.getElementById("searchInput").value.toLowerCase();
  const table = document.getElementById("mytable");
  const rows = Array.from(table.querySelectorAll("tbody tr"));

  // Filter
  let filteredRows = rows.filter(row =>
    row.textContent.toLowerCase().includes(input)
  );

  // Paginate
  rows.forEach(row => row.classList.add("hidden"));
  const start = (currentPage - 1) * rowsPerPage;
  const end = start + rowsPerPage;
  filteredRows.slice(start, end).forEach(row => row.classList.remove("hidden"));

  // Pagination Controls
  const totalPages = Math.ceil(filteredRows.length / rowsPerPage);
  const paginationDiv = document.getElementById("pagination");
  paginationDiv.innerHTML = "";
  for (let i = 1; i <= totalPages; i++) {
    let btn = document.createElement("button");
    btn.innerText = i;
    btn.style.marginRight = "5px";
    if (i === currentPage) btn.classList.add("active-page");
    btn.onclick = () => {
      currentPage = i;
      renderTable();
    };
    paginationDiv.appendChild(btn);
  }
}

document.addEventListener("DOMContentLoaded", () => {
  document.querySelectorAll("#mytable th").forEach((th, i) => {
    th.onclick = () => sortTable(i);
  });
  renderTable();
});
</script>


In [ ]:
#| echo: false
import pandas as pd
from IPython.display import HTML

df = pd.read_csv("cleaned_citydatademoCSV.csv")
HTML(df.to_html(index=False, table_id="mytable", classes="styled-table"))

<div id="table-controls" style="display: flex; justify-content: space-between; align-items: center; margin-top: 12px;">
  <div id="pagination"></div>

  <a href="cleaned_citydatademoCSV.csv" download="california_demographics.csv">
    <button
      class="city-button">
      Download CSV
    </button>
  </a>
</div>


## Interactive Map of Cities

Use the map below to explore demographic data by city. Select a variable and then hover a city to view population, income, and racial/ethnic breakdown.


In [ ]:
import geopandas as gpd
import pandas as pd
import json
import numpy as np
import ipywidgets as widgets
from ipyleaflet import Map, GeoJSON, WidgetControl, basemaps
from ipywidgets import HTML
import branca

# ─── 1) Load & filter CA places ─────────────────────────────────────────────
gdf = gpd.read_file("tl_2024_06_place.shp")
gdf = gdf[(gdf.LSAD == "25") & (gdf.STATEFP == "06")]
gdf["CITY_NAME"] = gdf.NAME.str.upper()

# ─── 2) Merge with your city stats ──────────────────────────────────────────
city_data = pd.read_csv("cleaned_citydatademoCSV.csv")
city_data["CITY_NAME"] = city_data["City Name"].str.upper()

vars_to_plot = [
    "Latino %","White %","Black %","Asian %",
    "Native %","PI %","Other %","Multiracial %",
    "Total Population","Median Household Income"
]

merged = (
    gdf[["CITY_NAME","geometry"]]
    .merge(city_data[["CITY_NAME"] + vars_to_plot], on="CITY_NAME")
)

# ─── 3) Pre‑serialize to GeoJSON once ────────────────────────────────────────
geojson_data = json.loads(merged.to_json())

# right after you filter gdf:
# choose a tolerance that preserves city‐shapes but drops lots of tiny vertices
gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.001, preserve_topology=True)

# then continue with your merge+to_json
geojson_data = json.loads(merged.to_json())

# ─── 4) Build base map with no‑street tiles ─────────────────────────────────
m = Map(
    center=(34.0, -118.2),
    zoom=9,
    basemap=basemaps.CartoDB.PositronNoLabels
)

# placeholders
geo_layer     = None
legend_ctrl   = None

# ─── 5) Helper to make a branca colormap for a variable ─────────────────────
def make_colormap(var):
    vmin, vmax = merged[var].min(), merged[var].max()
    return branca.colormap.LinearColormap(
        ['#fee5d9','#a50f15'], vmin=vmin, vmax=vmax, caption=var
    )

# ─── 6) Style‐function factory ───────────────────────────────────────────────
def style_fn_factory(var, cmap):
    def style_fn(feature):
        val = feature['properties'][var]
        return {
            'fillColor': cmap(val),
            'color':   'black',
            'weight':  0.3,
            'fillOpacity': 0.7
        }
    return style_fn

# ─── 7) Add one GeoJSON layer, initial var ─────────────────────────────────
initial_var = vars_to_plot[0]
cmap = make_colormap(initial_var)

geo_layer = GeoJSON(
    data=geojson_data,
    style_callback=style_fn_factory(initial_var, cmap),
    hover_style={'fillOpacity': 1.0}
)
m.add_layer(geo_layer)

# legend via HTML widget
legend = HTML(cmap._repr_html_())
legend_ctrl = WidgetControl(widget=legend, position='bottomright')
m.add_control(legend_ctrl)

# ─── 8) Dropdown & callback ────────────────────────────────────────────────
dropdown = widgets.Dropdown(options=vars_to_plot, value=initial_var, description='Variable:')
def on_var_change(change):
    var = change['new']
    new_cmap = make_colormap(var)
    geo_layer.style_callback = style_fn_factory(var, new_cmap)
    # update legend HTML
    legend.value = new_cmap._repr_html_()

dropdown.observe(on_var_change, names='value')

# ─── 9) Display ────────────────────────────────────────────────────────────
display(widgets.VBox([dropdown, m]))

## Heatmap: Demographic Variables by City


In [ ]:
#| echo: false
import pandas as pd
import plotly.express as px

# Load your data
df = pd.read_csv("cleaned_citydatademoCSV.csv")

# ✅ Only keep demographic percentage columns (no income)
cols_to_use = [
    "Latino %",
    "White %",
    "Black %",
    "Asian %",
    "Native %",
    "PI %",
    "Other %",
    "Multiracial %"
]


# Set city names as index
heatmap_df = df[["City Name"] + cols_to_use].set_index("City Name")

# Optional: sort by a demographic column (e.g. Latino %)
heatmap_df = heatmap_df.sort_values(by="Latino %", ascending=False)

# Create Plotly heatmap
fig = px.imshow(
    heatmap_df,
    color_continuous_scale="Reds",
    aspect="auto",
    labels=dict(x="Demographic Variable", y="City", color="Percent"),
    title="Demographic Composition by City"
)

fig.update_layout(
    height=800,
    margin=dict(l=100, r=20, t=60, b=20)
)

fig